# Submission of Calculation with Subsequent Results Query

In this demo, we will submit a calculation, check the status of the job and query the results after it is finished.

In [ ]:
from azure.quantum import Workspace
from azure.quantum.job import JobFailedWithResultsError

# insert connection string form Azure Portal Workspace Access Keys
connection_string = "" 
workspace = Workspace.from_connection_string(connection_string)

In [ ]:
print("Verifying access to DFT target.")

# To submit DFT jobs, we will be using the microsoft.dft target in the workspace.
target = workspace.get_targets("microsoft.dft")

In [ ]:
# First, let's define the molecular structure as xyz string

input_data = '''13
C6H6O
O            2.5032951000       0.0184294300       0.0413292100
C            1.1303877300       0.0093712000       0.0048092200
C            0.4231806300       1.2120715400       0.0534829600
C           -0.9683182400       1.2029759500      -0.0240138400
C           -1.6577837600      -0.0029850000      -0.1520961400
C           -0.9474329900      -1.2022693500      -0.2062671600
C            0.4442277200      -1.1989189200      -0.1290771400
H            2.7937631000      -0.0420078500       0.9582922000
H            0.9833812200       2.1439365800       0.1424090800
H           -1.5174320700       2.1452316000       0.0127800900
H           -2.7466629100      -0.0076855600      -0.2144254800
H           -1.4799101400      -2.1487692800      -0.3127229700
H            1.0205566600      -2.1237195000      -0.1807420600
'''

In [ ]:
# Secondly, let's give a name for the job.
job_name = 'C6H6O_spf'

Now we submit the calculations to MADFT service, looping over three different basis sets.

In [ ]:
# Next, we create a dictionary variable to specify the parameters for the DFT calculation. 
dft_input_params = {
  "tasks": [
    {
      "taskType": "spf", 
      "basisSet": { "name": 'def2-svpd'},
      "xcFunctional": { "name": "b3lyp", "gridLevel": 4 },
      "molecule": { "charge": 0, "multiplicity": 1 },
      "scf": { "method": "rks", "maxSteps": 100, "convergeThreshold": 1e-8 }
    }
  ]
}

# We are now ready to submit the Job using the target.submit call. It takes three parameters-
# 1. The input molecule in xyz format.
# 2. The DFT parameters that we declared above.
# 3. A friendly name to help identify the job in the Azure Portal later.

print("Submitting DFT job.")

job = target.submit(
    input_data = input_data,
    input_params = dft_input_params,
    name= job_name)
    
print("\nDFT job has been submitted.")
print(f"\nJob name: {job_name}")


We can retrieve information about a job through [Workspace.get_job](https://learn.microsoft.com/en-us/azure/quantum/optimization-job-management), and also query the results by filtering the job name with [Workspace.list_jobs](https://learn.microsoft.com/en-us/azure/quantum/optimization-job-management).

In [ ]:
# query the latest job that match the given name
job = workspace.list_jobs(name_match=job_name)[-1]

# refresh the job SAS for using the API
job.refresh()

# show the status of the job
print(f'Job: "{job_name}" is {job.details.status}')

In [ ]:
# read the results of the job
if job.details.status == 'Succeeded':
    results = job.get_results()
else:
    results = f'"{job_name}" is still {job.details.status}...'

# QCSchema Output
qcschema = results["results"][0]
# Return the force:
qcschema["return_result"]

## Output to QCSchema json file

In [ ]:
import json
n = job_name + "_output"
with open(n+".json", "w") as fp:
    json.dump(qcschema, fp)